In [4]:
import pandas as pd
import numpy as np
import torch
from importlib import reload
import sliced
from sliced import base
from sliced.base import slice_y, grouped_sum, unique_counts
import scipy
from scipy import linalg
from scipy.linalg import eigh
from scipy.linalg import qr
import cir
from cir import CIR
reload(cir)
# import CIR_
# from CIR_ import CIR
# reload(CIR_)
# import CIR2_
# from CIR2_ import CIR2
# reload(CIR2_)

import sys
import numpy
# numpy.set_printoptions(suppress=True, threshold=sys.maxsize)
import math
from scipy.linalg import eigh
from scipy.linalg import eig
from numpy.linalg import norm
import torch
import geoopt
from geoopt.optim import RiemannianSGD
from geoopt.manifolds import Stiefel
from scipy.linalg import solve
from scipy.linalg import cholesky

In [12]:
# from importlib import reload
import cir
from cir import CIR
reload(cir)
# import CIR_
# from CIR_ import CIR
# reload(CIR_)
import torch
X = pd.read_csv('foregroundX.csv')
Y = pd.read_csv('foregroundY.csv')
Xt = pd.read_csv('backgroundX.csv')
Yt = pd.read_csv('backgroundY.csv')

X = X.iloc[:, 1:]
Xt = Xt.iloc[:, 1:]
Y = Y.iloc[:, 1:]
Yt = Yt.iloc[:, 1:]

t = CIR(X, Y, Xt, Yt, 0.0008, 2, "SGPM")
torch.set_printoptions(threshold=sys.maxsize, sci_mode=False)
numpy.set_printoptions(suppress=True, threshold=sys.maxsize)
print(t)

O = pd.read_csv('output.csv')
O = pd.DataFrame(O)
O = O.iloc[:, 1:]
O = O.to_numpy()
output = O @ np.transpose(O)
output = torch.tensor(output)

# print(output)
torch.set_printoptions(threshold=sys.maxsize, sci_mode=False)
# print((torch.subtract(output, t)).detach().numpy())

# t1 = CIR2(X, Y, Xt, Yt, 0.0001, 2)
# print(np.subtract(output, t1))
# print(output)

/Users/lianama/anaconda3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/lianama/anaconda3/lib/python3.10/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


This is objective function:  -0.8678636583774679
[[ 0.05094815 -0.03853356  0.00128516 -0.10254975  0.00665907  0.02322163
  -0.01269177 -0.02106497  0.02623084 -0.00277898  0.00320138  0.04109763
  -0.01990238  0.02105945 -0.02718783  0.02358131  0.02388462  0.01449637
   0.02894327  0.02084906  0.02717871  0.0400795   0.01212898  0.00929399
  -0.01682459 -0.03509087 -0.01733766  0.03071755  0.02161179  0.00750067
   0.01182418  0.05646905 -0.01937342  0.01058055  0.01128038 -0.01252817
  -0.02502405 -0.00021731  0.00914192  0.04630337  0.03418135  0.00594455
  -0.00330876 -0.00849382  0.01015236  0.01902893  0.01100424 -0.01222461
  -0.00537131  0.01220724 -0.00655896  0.00387912  0.01350454 -0.03957663
   0.01821534 -0.04134423 -0.02492617 -0.00834154  0.01884838  0.00352348
  -0.01775217  0.00712246 -0.02262002  0.01884936  0.00541237 -0.01462867
  -0.05011933 -0.00904242 -0.00154581 -0.01419063  0.03078459  0.0379412
   0.01498465  0.02900766 -0.02762313  0.00310599  0.01255033]
 

In [63]:

def CIR(X, Y, Xt, Yt, a, d, opt_option):
    X = pd.DataFrame(X)
    Xt = pd.DataFrame(Xt)
    Y = pd.DataFrame(Y)
    Yt = pd.DataFrame(Yt)

    # n represents the rows of X. p represents the columns of X. m represent the rows of Xt.
    n = len(X)
    p = len(X.columns)
    m = len(Xt)

    if X.iloc[:, 0].equals(pd.Series(range(1, len(X) + 1))):
        raise ValueError("X should not have an index column")

    if Xt.iloc[:, 0].equals(pd.Series(range(1, len(Xt) + 1))):
        raise ValueError("Xt should not have an index column")

    if Y.iloc[:, 0].equals(pd.Series(range(1, len(Y) + 1))):
        raise ValueError("Y should not have an index column")

    if Yt.iloc[:, 0].equals(pd.Series(range(1, len(Yt) + 1))):
        raise ValueError("Yt should not have an index column")

    if len(Xt.columns) != p:
        raise ValueError("Xt should have the same number of columns as X")

    if len(Y) != n:
        raise ValueError("Y must have the same number of rows as X")

    if len(Yt) != m:
        raise ValueError("Yt must have the same number of rows as Xt")

    if not isinstance(d, int):
        raise TypeError("d parameter must be an integer")

    if d < 1:
        raise ValueError("d must be greater than or equal to 1")

    if a < 0:
        raise ValueError("a must be greater than or equal to 0")

    # Center the matrix X by subtracting the original matrix X by the column means of X
    col_mean_x = X.mean(axis=0)
    X = X - col_mean_x

    # Calculate the Covariance Matrix
    cov_matrix_X = X.cov()

    # Define H
    Y_unique_value = Y.nunique().item()
    if Y_unique_value == 2:
        H = 2
    elif 2 < Y_unique_value <= 10:
        H = Y_unique_value
    else:
        if d <= 2:
            H = 10
        else:
            H = 4

    # Define Ph, count the of ocurrence of y in each H interval.
    interval_Ph = pd.cut(Y.iloc[:, 0], bins=H)
    Ph = interval_Ph.value_counts().sort_index()

    # Find the mean: Take each row of X and average among each interval separately
    interval = np.linspace(np.min(Y), np.max(Y), num=H+1)
    mh = []
    for i in range(len(interval) - 1):
        mask = (interval[i] <= Y) & (Y <= interval[i+1])
        mask = mask.squeeze()  # convert to Series
        x_rows_mean = np.mean(X[mask.values], axis=0)
        mh.append(x_rows_mean)

    mh = np.array(mh)

    # Cov(E[X|Y])
    sigma_X = np.zeros((X.shape[1], X.shape[1]))
    for i in range(len(interval) - 1):
        mask = (interval[i] <= Y) & (Y <= interval[i+1])
        interval_mh = mh[i]
        outer_product = np.outer(interval_mh, interval_mh)

        if not np.isnan(outer_product).any():
            sigma_X += outer_product

    sigma_X = np.array(sigma_X)
    eigenvalues, eigenvectors = np.linalg.eigh(sigma_X)
    epsilon = 2 * abs(np.min(eigenvalues))
    sigma_X = sigma_X + epsilon * np.identity(p)

    cov_matrix_X = np.array(cov_matrix_X)

    # Generalized Eigenvalue Decomposition
    A = cov_matrix_X @ sigma_X @ cov_matrix_X
    B = cov_matrix_X @ cov_matrix_X
    eigenvalues, eigenvectors = eigh(cov_matrix_X, sigma_X)
    # print(eigenvectors)

    if a == 0:
        v = eigenvectors[:, :d]
        f_v = -1 * (np.trace(v.T @ A @ v @ np.linalg.inv(v.T @ B @ v)))
        return v, f_v
    # =================================================================
    # Background data and the case when a > 0

    # Center the data
    col_means_xt = Xt.mean(axis=0)
    Xt = Xt - col_means_xt

    # Covariance Matrix
    cov_matrix_Xt = Xt.cov()

    # Define Ht
    Yt_unique_value = Yt.nunique().item()
    if Yt_unique_value == 2:
        Ht = 2
    elif 2 < Yt_unique_value <= 10:
        Ht = Yt_unique_value
    else:
        if d <= 2:
            Ht = 10
        else:
            Ht = 4

    # Define Ph_t
    interval_Ph_t = pd.cut(Yt.iloc[:, 0], bins=Ht)
    Ph_t = interval_Ph_t.value_counts().sort_index()

    # Find the mean: Take each row of X and average among each interval separately
    interval_t = np.linspace(np.min(Yt), np.max(Yt), num=Ht+1)
    mh_t = []
    for i in range(len(interval_t) - 1):
        mask_t = (interval_t[i] <= Yt) & (Yt <= interval_t[i+1])
        mask = mask.squeeze()
        xt_rows_mean = np.mean(Xt[mask_t.values], axis=0)
        mh_t.append(xt_rows_mean)

    mh_t = np.array(mh_t)

    # Cov(E[Xt|Yt])
    sigma_Xt = np.zeros((Xt.shape[1], Xt.shape[1]))
    for i in range(len(interval_t) - 1):
        mask_t = (interval_t[i] <= Yt) & (Yt <= interval_t[i+1])
        interval_mh_t = mh_t[i]
        outer_product_t = np.outer(interval_mh_t, interval_mh_t)

        if not np.isnan(outer_product_t).any():
            sigma_Xt += outer_product_t

    sigma_Xt = np.array(sigma_Xt)
    eigenvalues_t, eigenvectors_t = np.linalg.eigh(sigma_Xt)
    epsilon_t = 2 * abs(np.min(eigenvalues_t))
    sigma_Xt = sigma_Xt + epsilon_t * np.identity(p)

    cov_matrix_Xt = np.array(cov_matrix_Xt)

    # Generalized Eigenvalue Decomposition
    At = cov_matrix_Xt @ sigma_Xt @ cov_matrix_Xt
    Bt = cov_matrix_Xt @ cov_matrix_Xt

    # Optimization on Manifold
    if opt_option == "geoopt":
        A = torch.tensor(A, dtype=torch.double)
        B = torch.tensor(B, dtype=torch.double)
        a = torch.tensor(a, dtype=torch.double)
        At = torch.tensor(At, dtype=torch.double)
        Bt = torch.tensor(Bt, dtype=torch.double)

        stiefel = geoopt.manifolds.Stiefel()
        torch.manual_seed(1)
        initial_point = torch.randn(p, d)
        initial_point, _ = torch.linalg.qr(initial_point)

        v = geoopt.ManifoldParameter(initial_point, manifold=stiefel)

        optimizer = RiemannianSGD([v], lr=1e-3, momentum=0.9)
        max_iterations = 10000

        for step in range(max_iterations):
            vt = v.clone()
            optimizer.zero_grad()
            cost = f_geoopt(A, B, a, v, At, Bt)
            gradient = grad_geoopt(A, B, a, v, At, Bt).to(torch.float)
            v.grad = gradient
            optimizer.step()
            vt_plus = v.clone()

            if stepExit(vt_plus, vt, cost, A, B, At, Bt, a):
                break

        output = v @ v.t()
        return output

    if opt_option == "SGPM":
        np.random.seed(2)
        v = np.random.rand(p, d)
        v, r = np.linalg.qr(v)

        output = SGPM(v, A, B, At, Bt, a)
        # return output
        return output @ np.transpose(output)


def f_geoopt(A, B, a, v, At, Bt):
    v = v.data
    v = v.to(torch.double)

    bv_inv = torch.inverse(v.t() @ B @ v)
    va = v.t() @ A @ v
    bv_t_inv = torch.inverse(v.t() @ Bt @ v)
    va_t = v.t() @ At @ v

    f_v = -torch.trace(va @ bv_inv) + a * torch.trace(va_t @ bv_t_inv)
    return f_v


def grad_geoopt(A, B, a, v, At, Bt):
    v = v.data
    v = v.to(torch.double)

    bv_inv = torch.inverse(v.t() @ B @ v)
    va = v.t() @ A @ v
    bv_t_inv = torch.inverse(v.t() @ Bt @ v)
    va_t = v.t() @ At @ v

    avb = A @ v @ bv_inv - B @ v @ bv_inv @ va @ bv_inv
    avb_t = At @ v @ bv_t_inv - Bt @ v @ bv_t_inv @ va_t @ bv_t_inv

    gradient = -2 * (avb - a * avb_t)
    return gradient


def stepExit(vt_plus, vt, cost, A, B, At, Bt, a) -> bool:
    xtol = 1e-6
    gtol = 1e-4
    ftol = 1e-12
    distance = torch.norm(vt_plus @ vt_plus.t() - vt @ vt.t(), 'fro')
    vt_plus_gradient = grad_geoopt(A, B, a, vt_plus, At, Bt)
    cost_vt_plus = f_geoopt(A, B, a, vt_plus, At, Bt)

    if distance < xtol:
        return True
    elif torch.norm(vt_plus_gradient, 'fro') < gtol:
        return True
    elif abs(cost_vt_plus - cost) < ftol:
        return True
    else:
        return False


def f(A, B, a, v, At, Bt):
    bv_inv = np.linalg.inv(np.transpose(v) @ B @ v)
    va = np.transpose(v) @ A @ v
    bv_t_inv = np.linalg.inv(np.transpose(v) @ Bt @ v)
    va_t = np.transpose(v) @ At @ v
    f_v = -np.matrix.trace(va @ bv_inv) + a * np.matrix.trace(va_t @ bv_t_inv)
    return f_v


def grad(A, B, a, v, At, Bt):
    bv_inv = np.linalg.inv(np.transpose(v) @ B @ v)
    va = np.transpose(v) @ A @ v
    bv_t_inv = np.linalg.inv(np.transpose(v) @ Bt @ v)
    va_t = np.transpose(v) @ At @ v

    avb = A @ v @ bv_inv - B @ v @ bv_inv @ va @ bv_inv
    avb_t = At @ v @ bv_t_inv - Bt @ v @ bv_t_inv @ va_t @ bv_t_inv

    gradient = -2 * (avb - a * avb_t)
    return gradient


def SGPM(X, A, B, At, Bt, a):
    X = np.array(X)
    n, k = X.shape

    xtol = 1e-6
    gtol = 1e-4
    ftol = 1e-12
    rho = 1e-4
    eta = 0.1
    gamma = 0.85
    tau = 1e-3
    STPEPS = 1e-10
    nt = 5
    mxitr = 1000
    alpha = 0.85
    record = 0
    projG = 1
    iscomplex = 0
    crit = np.ones((nt, 3))
    invH = True

    if k < n/2:
        invH = False
        eye2k = np.identity(2 * k)

    # Initial function value and gradient
    # Prepare for iterations
    F = f(A, B, a, X, At, Bt)
    G = grad(A, B, a, X, At, Bt)

    GX = np.transpose(G) @ X

    if invH:
        GXT = G @ np.transpose(X)
        H = GXT - np.transpose(GXT)
    else:
        if projG == 1:
            U = np.hstack((G, X))
            V = np.hstack((X, -G))
            VU = np.transpose(V) @ U
        elif projG == 2:
            GB = G - 0.5 * X @ (np.transpose(X) @ G)
            U = np.hstack((GB, X))
            V = np.hstack((X, -GB))
            VU = np.transpose(V) @ U
        VX = np.transpose(V) @ X

    dtX = G - X @ GX
    nrmG = np.linalg.norm(dtX, ord='fro')
    Q = 1
    Cval = F

    # main iteration
    F_eval = np.zeros((mxitr + 1, 1))
    Grad = np.zeros((mxitr + 1, 1))
    F_eval[0] = F
    Grad[0] = nrmG
    for itr in range(0, mxitr):
        XP, FP, dtXP, nrmGP = X, F, dtX, nrmG

        # scale step size
        nls = 1
        deriv = rho * (nrmG**2)

        while True:
            # Update Scheme
            if invH:
                if abs(alpha) < rho:    # Explicit Euler (Steepest Descent)
                    X = XP - tau * dtXP
                elif abs(alpha - 0.5) < rho:  # Crank-Nicolson
                    A = np.eye(n) + (tau * 0.5) * H
                    X = solve(
                        A, XP - (0.5 * tau) * dtXP, lower=False)
                elif abs(alpha - 1) < rho:  # Implicit EuLer
                    A = np.eye(n) + tau * H
                    X = solve(A, XP, lower=False)
                else:  # Convex Combination
                    A = np.eye(n) + (tau * alpha) * H
                    X = solve(
                        A, XP - ((1 - alpha) * tau) * dtXP, lower=False)

                if abs(alpha - 0.5) > rho:
                    XtX = np.transpose(X) @ X
                    L = cholesky(XtX, lower=False)
                    X = X @ np.linalg.inv(L)

            else:
                aa = solve(eye2k + (alpha * tau) * VU, VX)
                X = XP - U @ (tau * aa)

                if abs(alpha - 0.5) > rho:
                    XtX = np.transpose(X) @ X
                    L = cholesky(XtX, lower=False)
                    X = X @ np.linalg.inv(L)

            # calculate G, F
            F = f(A, B, a, X, At, Bt)
            G = grad(A, B, a, X, At, Bt)

            if F <= Cval - tau * deriv or nls >= 5:
                break

            tau = eta * tau
            nls += 1

        GX = np.transpose(G) @ X
        dtX = G - X @ GX
        nrmG = np.linalg.norm(dtX, ord='fro')

        F_eval[itr+1] = F
        # print(Grad)
        Grad[itr+1] = nrmG

        # Adaptive scaling matrix strategy
        if nrmG < nrmGP:
            if nrmG >= 0.5 * nrmGP:
                alpha = max(min(alpha * 1.1, 1), 0)
        else:
            alpha = max(min(alpha * 0.9, 0), 0.5)

        # Computing the Riemannian Gradient
        if invH:
            if abs(alpha) > rho:
                GXT = G @ np.transpose(X)
                H = GXT - np.transpose(GXT)
        else:
            if projG == 1:
                U = np.hstack((G, X))
                V = np.hstack((X, -G))
                VU = np.transpose(V) @ U
            elif projG == 2:
                GB = G - X @ (0.5 * np.transpose(GX))
                U = np.hstack((GB, X))
                V = np.hstack((X, -GB))
                VU = np.transpose(V) @ U
            VX = np.transpose(V) @ X

        # Compute the Alternate ODH step-size
        S = X - XP
        SS = np.sum(np.sum(np.multiply(S, S), axis=1), axis=0)

        XDiff = math.sqrt(SS / n)
        FDiff = abs(FP - F) / (abs(FP) + 1)

        Y = dtX - dtXP
        SY = abs(np.sum(np.sum(np.multiply(S, Y), axis=1), axis=0))

        if itr % 2 == 0:
            tau = SS / SY
        else:
            YY = np.sum(np.sum(np.multiply(Y, Y), axis=1), axis=0)
            tau = SY / YY

        tau = max(min(tau, 1e20), 1e-20)

        # Stopping Rules
        crit[itr, :] = [nrmG, XDiff, FDiff]

        mcrit = np.mean(crit[max(0, itr-nt+1):itr, :], axis=0)

        if (XDiff < xtol and FDiff < ftol) or nrmG < gtol or all(mcrit[1:3] < 10 * np.hstack((xtol, ftol))):
            if itr <= 2:
                ftol = 0.1 * ftol
                xtol = 0.1 * xtol
                gtol = 0.1 * gtol
            else:
                msg = "converge"
                break

        Qp = Q
        Q = gamma * Qp + 1
        Cval = (gamma * Qp * Cval + F) / Q

    F_eval = F_eval[0:itr, 0]
    Grad = Grad[0:itr, 0]

    if itr >= mxitr:
        msg = "exceed max iteration"

    feasi = np.linalg.norm(np.transpose(X) @ X - np.eye(k), 'fro')

    if feasi > 1e-13:
        L = np.linalg.cholesky(X.T @ X)
        X = X @ np.linalg.inv(L)
        F = f(A, B, a, X, At, Bt)
        G = grad(A, B, a, X, At, Bt)
        dtX = G - X @ GX
        nrmG = np.linalg.norm(dtX, 'fro')
        feasi = np.linalg.norm(X.T @ X - np.eye(k), 'fro')

    feasi = np.linalg.norm(X.T @ X - np.eye(k), 'fro')
    print("This is objective function: ", F)
    return X



X = pd.read_csv('foregroundX.csv')
Y = pd.read_csv('foregroundY.csv')
Xt = pd.read_csv('backgroundX.csv')
Yt = pd.read_csv('backgroundY.csv')

X = X.iloc[:, 1:]
Xt = Xt.iloc[:, 1:]
Y = Y.iloc[:, 1:]
Yt = Yt.iloc[:, 1:]

t = CIR(X, Y, Xt, Yt, 0.0001, 2, "SGPM")
numpy.set_printoptions(suppress=True, threshold=sys.maxsize)
print(t)

/Users/lianama/anaconda3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/lianama/anaconda3/lib/python3.10/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


IndexError: index 5 is out of bounds for axis 0 with size 5

In [16]:
X = pd.read_csv('foregroundX.csv')
Y = pd.read_csv('foregroundY.csv')
Xt = pd.read_csv('backgroundX.csv')
Yt = pd.read_csv('backgroundY.csv')

X = X.iloc[:, 1:]
Xt = Xt.iloc[:, 1:]
Y = Y.iloc[:, 1:]
Yt = Yt.iloc[:, 1:]

cov_X = X.cov()
cov_Xt = Xt.cov()

Y_unique_value = Y.nunique().item()
Yt_unique_value = Yt.nunique().item()
H = 8
Ht = 2
unique_y_vals, counts = unique_counts(Y)
cumsum_y = np.cumsum(counts)
n_y_values = unique_y_vals.shape[0]
slice_partition = np.hstack((0, cumsum_y))

slice_indicator = np.ones(Y.shape[0], dtype=int)
for j, (start_idx, end_idx) in enumerate(zip(slice_partition, slice_partition[1:])): 
    if j == len(slice_partition) - 2:
        slice_indicator[start_idx:] = j
    else:
        slice_indicator[start_idx:end_idx] = j

slice_counts = np.bincount(slice_indicator)
n, p = X.shape
Q, R = linalg.qr(X, mode='economic')
inv_idx = np.concatenate(([0], np.diff(slice_indicator).nonzero()[0] + 1))
Z_means = np.add.reduceat(Q, inv_idx) / np.sqrt(slice_counts.reshape(-1, 1))
M = np.dot(Z_means.T, Z_means)

cov_X = np.array(cov_X)

A = cov_X @ M @ cov_X
B = cov_X @ cov_X
"""-----------background------------"""
unique_yb_vals, countsB = unique_counts(Yt)
cumsum_yb = np.cumsum(countsB)
n_yb_values = unique_yb_vals.shape[0]
slice_partition_B = np.hstack((0, cumsum_yb))

slice_indicatorB = np.ones(Yt.shape[0], dtype=int)
for j, (start_idx, end_idx) in enumerate(zip(slice_partition_B, slice_partition_B[1:])): 
    if j == len(slice_partition_B) - 2:
        slice_indicatorB[start_idx:] = j
    else:
        slice_indicatorB[start_idx:end_idx] = j

slice_counts_B = np.bincount(slice_indicatorB)
m, k = Xt.shape
Qt, Rt = linalg.qr(Xt, mode='economic')
inv_idx_B = np.concatenate(([0], np.diff(slice_indicatorB).nonzero()[0] + 1))
Z_means_B = np.add.reduceat(Qt, inv_idx_B) / np.sqrt(slice_counts_B.reshape(-1, 1))
Mt = np.dot(Z_means_B.T, Z_means_B)

cov_Xt = np.array(cov_Xt)

At = cov_Xt @ Mt @ cov_Xt
Bt = cov_Xt @ cov_Xt


def f(A, B, a, v, At, Bt):
    bv_inv = np.linalg.inv(np.transpose(v) @ B @ v)
    va = np.transpose(v) @ A @ v
    bv_t_inv = np.linalg.inv(np.transpose(v) @ Bt @ v)
    va_t = np.transpose(v) @ At @ v
    f_v = -np.matrix.trace(va @ bv_inv) + a * np.matrix.trace(va_t @ bv_t_inv)
    return f_v


def grad(A, B, a, v, At, Bt):
    bv_inv = np.linalg.inv(np.transpose(v) @ B @ v)
    va = np.transpose(v) @ A @ v
    bv_t_inv = np.linalg.inv(np.transpose(v) @ Bt @ v)
    va_t = np.transpose(v) @ At @ v

    avb = A @ v @ bv_inv - B @ v @ bv_inv @ va @ bv_inv
    avb_t = At @ v @ bv_t_inv - Bt @ v @ bv_t_inv @ va_t @ bv_t_inv

    gradient = -2 * (avb - a * avb_t)
    return gradient

def SGPM(X, A, B, At, Bt, a):
    X = np.array(X)
    n, k = X.shape

    xtol = 1e-6
    gtol = 1e-4
    ftol = 1e-12
    rho = 1e-4
    eta = 0.1
    gamma = 0.85
    tau = 1e-3
    STPEPS = 1e-10
    nt = 5
    mxitr = 1000
    alpha = 0.85
    record = 0
    projG = 2
    iscomplex = 0
    crit = np.ones((mxitr, 3))
    invH = True

    if k < n/2:
        invH = False
        eye2k = np.identity(2 * k)

    # Initial function value and gradient
    # Prepare for iterations
    F = f(A, B, a, X, At, Bt)
    G = grad(A, B, a, X, At, Bt)

    GX = np.transpose(G) @ X

    if invH:
        GXT = G @ np.transpose(X)
        H = GXT - np.transpose(GXT)
    else:
        if projG == 1:
            U = np.hstack((G, X))
            V = np.hstack((X, -G))
            VU = np.transpose(V) @ U
        elif projG == 2:
            GB = G - 0.5 * X @ (np.transpose(X) @ G)
            U = np.hstack((GB, X))
            V = np.hstack((X, -GB))
            VU = np.transpose(V) @ U
        VX = np.transpose(V) @ X

    dtX = G - X @ GX
    nrmG = np.linalg.norm(dtX, ord='fro')
    Q = 1
    Cval = F

    # main iteration
    F_eval = np.zeros((mxitr + 1, 1))
    Grad = np.zeros((mxitr + 1, 1))
    F_eval[0] = F
    Grad[0] = nrmG
    for itr in range(0, mxitr):
        XP, FP, dtXP, nrmGP = X, F, dtX, nrmG

        # scale step size
        nls = 1
        deriv = rho * (nrmG**2)

        while True:
            # Update Scheme
            if invH:
                if abs(alpha) < rho:    # Explicit Euler (Steepest Descent)
                    X = XP - tau * dtXP
                elif abs(alpha - 0.5) < rho:  # Crank-Nicolson
                    A = np.eye(n) + (tau * 0.5) * H
                    X = solve(
                        A, XP - (0.5 * tau) * dtXP, lower=False)
                elif abs(alpha - 1) < rho:  # Implicit EuLer
                    A = np.eye(n) + tau * H
                    X = solve(A, XP, lower=False)
                else:  # Convex Combination
                    A = np.eye(n) + (tau * alpha) * H
                    X = solve(
                        A, XP - ((1 - alpha) * tau) * dtXP, lower=False)

                if abs(alpha - 0.5) > rho:
                    XtX = np.transpose(X) @ X
                    L = cholesky(XtX, lower=False)
                    X = X @ np.linalg.inv(L)

            else:
                aa = solve(eye2k + (alpha * tau) * VU, VX)
                X = XP - U @ (tau * aa)

                if abs(alpha - 0.5) > rho:
                    XtX = np.transpose(X) @ X
                    L = cholesky(XtX, lower=False)
                    X = X @ np.linalg.inv(L)

            # calculate G, F
            F = f(A, B, a, X, At, Bt)
            G = grad(A, B, a, X, At, Bt)

            if F <= Cval - tau * deriv or nls >= 5:
                break

            tau = eta * tau
            nls += 1

        GX = np.transpose(G) @ X
        dtX = G - X @ GX
        nrmG = np.linalg.norm(dtX, ord='fro')

        F_eval[itr+1] = F
        # print(Grad)
        Grad[itr+1] = nrmG

        # Adaptive scaling matrix strategy
        if nrmG < nrmGP:
            if nrmG >= 0.5 * nrmGP:
                alpha = max(min(alpha * 1.1, 1), 0)
        else:
            alpha = max(min(alpha * 0.9, 0), 0.5)

        # Computing the Riemannian Gradient
        if invH:
            if abs(alpha) > rho:
                GXT = G @ np.transpose(X)
                H = GXT - np.transpose(GXT)
        else:
            if projG == 1:
                U = np.hstack((G, X))
                V = np.hstack((X, -G))
                VU = np.transpose(V) @ U
            elif projG == 2:
                GB = G - X @ (0.5 * np.transpose(GX))
                U = np.hstack((GB, X))
                V = np.hstack((X, -GB))
                VU = np.transpose(V) @ U
            VX = np.transpose(V) @ X

        # Compute the Alternate ODH step-size
        S = X - XP
        SS = np.sum(np.sum(np.multiply(S, S), axis=1), axis=0)

        XDiff = math.sqrt(SS / n)
        FDiff = abs(FP - F) / (abs(FP) + 1)

        Y = dtX - dtXP
        SY = abs(np.sum(np.sum(np.multiply(S, Y), axis=1), axis=0))

        if itr % 2 == 0:
            tau = SS / SY
        else:
            YY = np.sum(np.sum(np.multiply(Y, Y), axis=1), axis=0)
            tau = SY / YY

        tau = max(min(tau, 1e20), 1e-20)

        # Stopping Rules
        crit[itr, :] = [nrmG, XDiff, FDiff]

        mcrit = np.mean(crit[max(0, itr-nt+1):itr, :], axis=0)
        # mcrit = np.mean(crit[itr - min(nt, itr) + 1:itr, :], axis=0)
        # print("Iteration:", itr)
        # print("size of crit", crit.shape)
        # print("size of mcrit", mcrit.shape)

        if (XDiff < xtol and FDiff < ftol) or nrmG < gtol or all(mcrit[1:3] < 10 * np.hstack((xtol, ftol))):
            if itr <= 2:
                ftol = 0.1 * ftol
                xtol = 0.1 * xtol
                gtol = 0.1 * gtol
            else:
                msg = "converge"
                break

        Qp = Q
        Q = gamma * Qp + 1
        Cval = (gamma * Qp * Cval + F) / Q

    F_eval = F_eval[0:itr, 0]
    Grad = Grad[0:itr, 0]

    if itr >= mxitr:
        msg = "exceed max iteration"

    feasi = np.linalg.norm(np.transpose(X) @ X - np.eye(k), 'fro')

    if feasi > 1e-13:
        L = np.linalg.cholesky(X.T @ X)
        X = X @ np.linalg.inv(L)
        F = f(A, B, a, X, At, Bt)
        G = grad(A, B, a, X, At, Bt)
        dtX = G - X @ GX
        nrmG = np.linalg.norm(dtX, 'fro')
        feasi = np.linalg.norm(X.T @ X - np.eye(k), 'fro')

    feasi = np.linalg.norm(X.T @ X - np.eye(k), 'fro')
    return X

np.random.seed(2)
v = np.random.rand(p, 2)
v, r = np.linalg.qr(v)

output = SGPM(v, A, B, At, Bt, 0.0001)
        # return output
numpy.set_printoptions(suppress=True, threshold=sys.maxsize)
print(output @ np.transpose(output))


/Users/lianama/anaconda3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/lianama/anaconda3/lib/python3.10/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


[[ 0.14280525  0.03267481  0.02361189 -0.0843239  -0.00316438  0.02856273
  -0.02338388  0.0077746   0.06109031 -0.00979953  0.02032333  0.0185424
  -0.01397438  0.01095694 -0.04919587  0.04738624  0.10897747 -0.00032168
  -0.03406363  0.04758877  0.023619    0.08386394  0.02332875 -0.03770264
  -0.01024531 -0.11635897 -0.02678977  0.03336391  0.03830091  0.02865948
   0.0873574   0.00623701  0.00057852  0.03637378  0.06970999 -0.04202002
  -0.06705653  0.00335459 -0.0079792   0.02479591  0.02535769  0.02531976
  -0.02126961 -0.00356086 -0.03605121  0.0207118   0.03088454 -0.0055453
  -0.02023901  0.02924706 -0.02297593  0.01803724  0.03658051 -0.03008568
   0.0367677  -0.02641956 -0.0372077   0.00291026  0.03320894  0.01880077
   0.04153402  0.01841618 -0.00736536  0.00109689  0.02026728 -0.00548933
  -0.10510934  0.01994642  0.01952294 -0.01348174  0.05798056  0.06345901
   0.02000222  0.03882891 -0.01805676 -0.00258016  0.03645531]
 [ 0.03267481  0.0156197   0.00984971 -0.01417043 -

In [4]:
interval_Ph = pd.cut(Y.iloc[:, 0], bins=H)
Ph = interval_Ph.value_counts().sort_index()
interval = np.linspace(np.min(Y), np.max(Y), num=H+1)
mh = []
for i in range(len(interval) - 1):
    mask = (interval[i] <= Y) & (Y <= interval[i+1])
    mask = mask.squeeze()  # convert to Series
    x_rows_mean = np.mean(X[mask.values], axis=0)
    mh.append(x_rows_mean)

# mh = np.array(mh)
# print(mh)

n_obs = np.floor(Y.shape[0]/H)
n_samples_seen = 0 
slice_partition = [0]

while n_samples_seen < Y.shape[0] - 2: 
    slice_start = np.where(cumsum_y >= n_samples_seen + n_obs)[0]
    if slice_start.shape[0] == 0:
        slice_start = cumsum_y.shape[0] - 1
    else: 
        slice_start = slice_start[0]

    n_samples_seen = cumsum_y[slice_start]
    slice_partition.append(n_samples_seen)
    print(n_samples_seen)

Z = np.sqrt(n) * Q
Z = Q[np.argsort(Y), :]

M = np.outer(Z_means, Z_means)
M = np.dot(Z_means.T, Z_means)
print(M)

sigma_X = np.zeros((X.shape[1], X.shape[1]))
for i in range(len(interval) - 1):
    mask = (interval[i] <= Y) & (Y <= interval[i+1])
    interval_mh = mh[i]
    outer_product = np.outer(interval_mh, interval_mh)

    if not np.isnan(outer_product).any():
        sigma_X += outer_product
print(sigma_X)
M = np.dot(Z_means.T, Z_means) / n              # covariance matrix 
print(M)

print(M.shape)
A = cov_X @ M @ cov_X
B = M @ M
eigenvalues, eigenvectors = eigh(cov_X, M)
v = eigenvectors[:, :2]
f_v = -1 * (np.trace(v.T @ A @ v @ np.linalg.inv(v.T @ B @ v)))

In [33]:
import CIR_
from CIR_ import CIR
O = pd.read_csv('output.csv')
O = pd.DataFrame(O)
O = O.iloc[:, 1:]
O = O.to_numpy()
output = O @ np.transpose(O)

output = torch.from_numpy(output)
t = CIR(X, Y, Xt, Yt, 0.0001, 2, "SGPM")
torch.set_printoptions(sci_mode=False)
print(t)

# print(torch.subtract(output, t))

# t1 = CIR2(X, Y, Xt, Yt, 0.0001, 2)
# print(np.subtract(output, t1))
# print(output)

/Users/lianama/anaconda3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/lianama/anaconda3/lib/python3.10/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


[[ 0.00056911 -0.00092996 -0.00046195  0.00707742 -0.00046091 -0.00133518
  -0.00240116 -0.00296885  0.00000897  0.00074709 -0.00283696 -0.00053871
  -0.00231333 -0.00016833 -0.00077488 -0.00448904  0.00598545  0.00852505
  -0.00226466  0.00006527 -0.00024297  0.00273696 -0.00106332  0.00064241
  -0.00193515 -0.00539584 -0.00369038  0.00130631  0.00186573 -0.00260053
  -0.00094098  0.00017602  0.00014511 -0.00013981 -0.00235425 -0.00425894
  -0.00371232 -0.00367587  0.0001926  -0.0012231  -0.00039227 -0.00154522
   0.00307536 -0.00174763  0.00308391  0.00099182 -0.00156348 -0.00138809
  -0.00150921  0.00093397 -0.00026884 -0.00150835 -0.00066834 -0.00455843
  -0.00079495 -0.00437959 -0.00324011 -0.00324545 -0.00050777 -0.00165368
  -0.00306756 -0.00123651 -0.00086526 -0.00530296 -0.00024778  0.00047987
  -0.00531313 -0.00199856 -0.00355265 -0.00076705  0.00081614  0.00072745
   0.0007031   0.00167644 -0.0033594   0.0010536   0.0053375 ]
 [-0.00092996  0.01999821 -0.00443902  0.02283838

In [6]:
# t = pd.read_csv('test-fX.csv')
# t = pd.DataFrame(t)
# if t.iloc[:, 0].dtypes != t.iloc[:, 1].dtypes:
#     t = t.iloc[:, 1:]

In [ ]:
# t = CIR(X, Y, Xt, Yt, 0.0001, 2)
# print(torch.subtract(output, t))

# output = torch.from_numpy(output)
# print("Correct output....")
# print(output)

# # n = pd.read_csv('test - noIndex.csv')

# X = pd.read_csv('test - fX.csv')
# Y = pd.read_csv('test - fY.csv')
# Xt = pd.read_csv('test - bX.csv')
# Yt = pd.read_csv('test - fY.csv')